In [9]:
# import numpy as np
# import pandas as pd
# from tqdm.auto import tqdm
import os
# import matplotlib.pyplot as plt
# # pd.set_option('display.max_columns', None)
# from matplotlib import rc
# rc('text',usetex=True)
# rc('text.latex', preamble='\\usepackage{color}')

In [12]:
target_name = 'random_5_15_s5'
sym_graver_bin = '../build/sym_hilbert'

In [13]:
os.makedirs(f'mats_{target_name}', exist_ok=True)
problem_list = []
for n in range(100):
    out_fname = f'mats_{target_name}/problem{n:03}.mat'
    os.remove(out_fname) if os.path.exists(out_fname) else None
    if not os.path.exists(out_fname):
        cmd = f'{sym_graver_bin} -unit-test-generate-matrix 5 15 {out_fname}'
        print(cmd)
        os.system(cmd)

    problem_list.append(out_fname.split('.')[0])

with open(f'models_{target_name}.txt', 'w') as f:
    for m in problem_list:
        f.write(m + '\n')

../build/sym_hilbert -unit-test-generate-matrix 5 15 mats_random_5_15_s5/problem000.mat

max_Zgen_val = 4, freq=528
H4:
 0 -3  0 -1 0 -2  2  0  2  0 1 0 0  0 0 
-3  1 -1  0 0  2 -2 -1 -4 -1 0 1 0  0 0 
-3  0 -1  0 0  0  0 -1  0  1 0 0 1  0 0 
 3 -3  1 -3 1 -2 -1  1  2 -2 0 0 0 -1 0 
 0  0  0  1 0  0  0  0  0  1 0 0 0  1 1 

H5:
0 0 0 0  2 1 0 0 0 0  2 -2  0 0  0 
0 0 0 0  1 0 1 0 0 0 -2  2  0 0  0 
0 0 0 0 -1 0 0 1 0 0  0  1  1 0  0 
0 0 0 0 -2 0 0 0 1 0 -2  4  0 0  0 
0 0 0 0  2 0 0 0 0 1  0  1 -1 0 -1 
0 1 0 0  3 0 0 0 0 0  3 -1  0 0  0 
0 0 1 0 -1 0 0 0 0 0  0  1  1 0  0 
0 0 0 1  3 0 0 0 0 0  1  0  0 0 -1 
0 0 0 0  1 0 0 0 0 0  0  0  0 1 -1 
1 0 0 0 -3 0 0 0 0 0  0  3  3 0  0 

../build/sym_hilbert -unit-test-generate-matrix 5 15 mats_random_5_15_s5/problem001.mat


Unknown option: 5
Unknown option: 5



max_Zgen_val = 4, freq=619
H4:
0 0  -6 0 0 -3 1 -5 3  -6 -4 -1 -1  3 0 
0 0   4 1 0  5 0  7 0   4  2  1  5 -6 0 
1 0   8 0 0  7 0  8 0   6  4  2  2 -6 0 
0 1   4 0 0  3 0  4 0   3  2  1  1 -3 0 
0 0 -12 0 0 -9 0 -9 0 -11 -6 -3  0  9 1 

H5:
-1 0 0 -2 0 1  0 0 0 0 0 -3 0 0  0 
 0 0 1  0 0 0  2 0 0 0 0 -4 0 0  0 
 0 0 0 -3 0 0  1 1 0 0 0 -4 0 0 -3 
 0 0 0  0 0 0 -3 0 1 0 0  0 0 0  0 
 0 0 0 -1 0 0  3 0 0 1 0 -3 0 0  2 
 0 0 0  0 0 0  2 0 0 0 1 -2 0 0  0 
 0 0 0  0 1 0  0 0 0 0 0  0 0 0  0 
 0 0 0 -4 0 0  0 0 0 0 0 -1 1 0 -3 
 0 0 0  3 0 0  0 0 0 0 0  3 0 1  0 
 2 1 0  1 0 0 -1 0 0 0 0 -1 0 0 -3 

../build/sym_hilbert -unit-test-generate-matrix 5 15 mats_random_5_15_s5/problem002.mat

max_Zgen_val = 4, freq=644
H4:
 2 0 0 -3  0 0 0  0  0  0 1  0  0  0 -3 
 3 0 0 -3  3 1 0 -2 -2 -4 0  2 -2 -2  0 
-2 1 0  3 -2 0 0  1  3  2 0 -1  3  0  3 
 2 0 1 -1  3 0 0  0 -2 -2 0 -1  1 -2 -2 
-4 0 0  3 -3 0 1  2  3  2 0 -1  1 -1  2 

H5:
0  2 -3 0 1 -3  3 0 0 0  0 0 0 0 0 
0 -3  1 1 0  3 -3 0 0 0  3 0 0 

Unknown option: 5
Unknown option: 5
Unknown option: 5



max_Zgen_val = 4, freq=617
H4:
 8  5 0 -1 -17 4 0 0  1 0 1 -6 -12 0 0 
 9  6 0 -2 -22 4 0 1  1 0 0 -7 -14 0 0 
 1 -1 1  0  -2 0 0 0 -2 1 0 -1  -2 0 0 
-1  0 0  0   2 0 0 0  0 0 0  1   2 1 0 
 4  3 0 -1  -9 2 0 0  1 0 0 -3  -6 0 1 

H5:
0 0  0  2 0 1 0  0 0 0 -2 0 0 0 0 
0 0  0  0 0 0 1  0 0 0  0 0 0 0 0 
2 0  0 -1 1 0 0  2 0 0  0 0 0 0 0 
0 0  2  1 0 0 0  1 1 0  0 0 0 0 0 
0 0 -1  0 0 0 0  0 0 1  0 0 0 0 0 
0 1  1  3 0 0 0  0 0 0 -2 0 0 0 0 
1 0  0  1 0 0 0  0 0 0 -1 1 0 0 0 
2 0  0  2 0 0 0  0 0 0 -2 0 1 0 0 
1 0 -1  4 0 0 0 -1 0 0 -4 0 0 1 0 
0 0  0  1 0 0 0  2 0 0  1 0 0 0 1 

../build/sym_hilbert -unit-test-generate-matrix 5 15 mats_random_5_15_s5/problem004.mat

max_Zgen_val = 5, freq=650
H4:
-2 0 0 0 -3  0 -3 -1  0 -2 1 0  3 -1 0 
 0 0 0 1  0  0 -5  0  2  1 0 0  3 -4 0 
 0 1 0 0  0 -1 -5  3  0  1 0 0  2 -5 0 
 2 0 1 0 -2 -1  0 -2 -2  0 0 1  1 -1 0 
-2 0 0 0  0 -1 -4  1  0  1 0 1 -1 -5 1 

H5:
0  0  2  0 1  0 0 0 0 0  3 0 0 0 0 
0  1 -4  5 0 -4 1 0 0 0  3 0 0 0 0 
0 -2  3  0 0  1

...

Unknown option: 5
Unknown option: 5
Unknown option: 5



max_Zgen_val = 4, freq=643
H4:
0 -2 0  0  1 0  0  0  0  0 1 3 0 0 0 
2  3 1 -2  4 0 -2  1 -2 -2 0 0 0 0 0 
3 -2 0  1 -3 0  3 -3  3  2 0 0 1 0 0 
3  0 0 -1  0 0  0  0  1 -2 0 0 0 1 0 
0  0 0  0 -2 1  0 -3  2  2 0 0 0 0 1 

H5:
0 1 -3 0 0  0 0 0 0 0  2 0  2  0 0 
0 0  2 0 0  0 1 0 0 0  0 0 -3  0 0 
0 0 -1 0 0  3 0 1 0 0  0 0  3  0 0 
0 0  2 0 0 -2 0 0 1 0  0 0 -3 -1 0 
0 0  2 0 0 -2 0 0 0 1  0 0 -2  2 0 
1 0 -2 0 0  0 0 0 0 0  0 0 -3 -3 0 
0 0  0 0 0  0 0 0 0 0 -3 1  0  0 0 
0 0 -4 0 1  2 0 0 0 0 -1 0  3  0 0 
0 0  2 1 0  0 0 0 0 0  0 0 -1  1 0 
0 0  0 0 0 -1 0 0 0 0  0 0  0  0 1 

../build/sym_hilbert -unit-test-generate-matrix 5 15 mats_random_5_15_s5/problem095.mat

max_Zgen_val = 4, freq=792
H4:
-3  0 0  0  0 2  0 0  0 0 1  0  0 0 0 
 3 -2 0  0  1 0  2 1 -1 2 0  3  1 0 0 
-3  2 1  1  0 1 -1 0  0 0 0  2 -1 0 0 
 0  0 0  0  1 0 -2 0 -1 3 0 -2  2 1 0 
-1  0 0 -3 -1 2 -2 0 -1 1 0 -2 -1 0 1 

H5:
0 0 -1 0  2 1 0 -2 0 0 -2 0 0 -2 0 
0 0  1 0 -2 0 1  0 0 0  0 0 0  4 0 
0 0 -1 1 -3 0 0  3 0

Unknown option: 5
Unknown option: 5
Unknown option: 5



max_Zgen_val = 5, freq=601
H4:
 2 0  0 0 0  4 1  4  5 -1  1 0 -2  5 0 
 2 0  0 0 0  3 0  3  3 -3  0 1 -3  2 0 
-3 0  0 0 1 -1 0 -2 -4 -2 -1 0  2 -3 0 
 3 1  0 0 0 -1 0  0  0  3  1 0  2  2 0 
 3 0 -3 0 0  1 0  3  3  2  0 0 -2  0 1 

H5:
0  1 0 0  1 1 -4 0 0 0 0 -3 0 0 -1 
0  0 0 1  0 0  0 0 0 0 0  0 0 0  0 
0  0 0 0  2 0 -4 1 0 0 0 -3 0 0 -3 
0  0 0 0  4 0 -5 0 1 0 0 -3 0 0 -3 
0 -3 0 0  2 0  1 0 0 1 0  3 0 0 -2 
0 -1 0 0  1 0 -1 0 0 0 1  0 0 0  0 
0  0 1 0  0 0  0 0 0 0 0  0 0 0  3 
0 -2 0 0 -2 0  2 0 0 0 0  3 1 0  2 
0 -2 0 0  3 0 -5 0 0 0 0 -2 0 1  0 
1 -3 0 0  3 0 -2 0 0 0 0 -2 0 0 -3 

../build/sym_hilbert -unit-test-generate-matrix 5 15 mats_random_5_15_s5/problem098.mat

max_Zgen_val = 5, freq=804
H4:
0  0 1  2  2 0  0 -1 0 0  0  1 0 -3  3 
1 -1 0  4  3 0 -4  0 1 0  4  3 0 -3  5 
0  4 0 -4  0 0  4  0 1 0 -5 -4 1 -3  1 
0  3 0 -2 -1 0  2 -2 0 1 -3 -1 0  0 -2 
0  0 0  2  3 1 -3 -2 2 0 -3 -1 0 -3  3 

H5:
-3 0 -2 0 1 -3 0 0 0  1 0 0  0 0 0 
 4 0  0 0 0  3 1 0 0 -2 0 0 -4 0 0 
 0 0 

Unknown option: 5
Unknown option: 5
Unknown option: 5
